In [14]:
import os 
os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['OPENAI_API_KEY'] = '...'



## Logical Routing

### Detailed Explanation

1. **Imports**:
    ```python
    from pydantic import BaseModel, Field
    from typing import Literal
    ```
    - `BaseModel`: A base class provided by Pydantic. When you inherit from this class, your class will automatically gain Pydantic's data validation features.
    - `Field`: A function from Pydantic used to provide additional metadata and validation for model fields.
    - `Literal`: A type hint from Python's `typing` module that restricts a variable to a specific set of values.

2. **Class Definition**:
    ```python
    class RouteQuery(BaseModel):
        '''Route a user query to the most relevant datasource'''
    ```
    - `class RouteQuery(BaseModel)`: This defines a new class `RouteQuery` that inherits from `BaseModel`. By inheriting from `BaseModel`, the class will automatically support Pydantic's features such as data validation, parsing, and serialization.
    - `'''Route a user query to the most relevant datasource'''`: This is a docstring that provides a brief description of the class.

3. **Field Definition**:
    ```python
    datasource: Literal['Python_docs', 'js_docs', 'golang_docs'] = Field(
        ...,
        description='given a user question choos which datasoure would be most relevant for answering their question',
    )
    ```
    - `datasource: Literal['Python_docs', 'js_docs', 'golang_docs']`: This line defines a class attribute `datasource` which must be one of the specified literal values ('Python_docs', 'js_docs', 'golang_docs'). Using `Literal` restricts the allowed values for this attribute, ensuring that it can only be set to one of these three strings.
    - `= Field(...)`: The `Field` function is used to provide additional metadata and validation rules for the `datasource` attribute.
    - `...`: In Pydantic, `...` is used as a placeholder for the default value of the datasource
    - `description='given a user question choos which datasoure would be most relevant for answering their question'`: This is an additional metadata attribute provided to `Field` that describes the purpose of the `datasource` field. This description can be used for documentation purposes or for generating user interfaces.


In [15]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

In [18]:
result = router.invoke("react code")

In [21]:
def choose_route(result):
    if 'python_docs' in result.datasource.lower():
        ### Logic here
        return 'python_docs'

    elif 'js_docs' in result.datasource.lower():
        ### Logic here
        return 'js_docs'

    else:
        return 'golang_docs'


from langchain_core.runnables import RunnableLambda
full_chain = router|RunnableLambda(choose_route)

In [22]:
full_chain.invoke('what is pydantic')

'python_docs'

## Semantic Routing

In [49]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(dict):
    # Embed question
    query_embedding = embeddings.embed_query(dict)
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print(most_similar)
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
   RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

print(chain.invoke("87+45*12"))

You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}
Using MATH
First, we need to follow the order of operations, which is parentheses, exponents, multiplication and division (from left to right), and addition and subtraction (from left to right).

So, first we multiply 45 by 12:
45 * 12 = 540

Then, we add 87 to the result:
87 + 540 = 627

Therefore, the answer to 87 + 45 * 12 is 627.


In [54]:
PromptTemplate.from_template(math_template).invoke({"query":"87+45*12"})
PromptTemplate.from_template(math_template).invoke("87+45*12")

StringPromptValue(text='You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, answer the component parts, and then put them together to answer the broader question.\n\nHere is a question:\n87+45*12')

   The cosine_similarity function takes two arrays as 
   input and returns an array of similarity scores. In this case, 
   the similarity scores are stored in the similarity variable.

  The argmax function is used to find the index of
  the maximum similarity score in the similarity array. 

# Query Construction

Many vectorstores contain metadata fields.

This makes it possible to filter for specific chunks based on metadata.

Let's look at some example metadata we might see in a database of YouTube transcripts.

Docs:

https://python.langchain.com/docs/use_cases/query_analysis/techniques/structuring

In [61]:
from langchain_community.document_loaders import YoutubeLoader

docs = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=pbAd8O1Lvm4", add_video_info=True
).load()

docs[0]

Document(page_content="hi this is Lance from Lang chain I'm going to be talking about using Lang graph to build a diverse and sophisticated rag flows so just to set the stage the basic rag flow you can see here starts with a question retrieval of relevant documents from an index which are passed into the context window of an llm for generation of an answer grounded in the ret documents so that's kind of the basic outline and we can see it's like a very linear path um in practice though you often encounter a few different types of questions like when do we actually want to retrieve based upon the context of the question um are the retrieve documents actually good or not and if they're not good should we discard them and then how do we loot back and retry retrieval with for example an improved question so these types of questions motivate an idea of active rag which which is a process where an llm actually decides when and where to retrieve based upon like existing retrievals or existing

In [63]:
docs[0].metadata

{'source': 'pbAd8O1Lvm4',
 'title': 'Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'description': 'Unknown',
 'view_count': 19116,
 'thumbnail_url': 'https://i.ytimg.com/vi/pbAd8O1Lvm4/hq720.jpg',
 'publish_date': '2024-02-07 00:00:00',
 'length': 1058,
 'author': 'LangChain'}

Let’s assume we’ve built an index that:

Allows us to perform unstructured search over the `contents` and `title` of each document
And to use range filtering on `view count`, `publication date`, and `length`.
We want to convert natural language into structured search queries.

We can define a schema for structured search queries.

In [75]:
import datetime
from typing import Literal,Optional, Tuple
from langchain_core.pydantic_v1 import BaseModel,Field

class TutorialSearch(BaseModel):
    '''Search over database of tutorial videos about a software library.'''

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )

    title_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to video titles."
            "Should be succinct and only include key words that could be in a video"
            "title"
        ),
    )

    min_view_count: Optional[int] = Field(
        None,
        description= 'Minimum view count filter, inclusive. Only use of explecitely specified',
    )

    max_view_count: Optional[int] = Field(
        None,
        description= 'Maximum view count filter, inclusive. Only use if explecitely specified',
    )

    earliest_publish_date: Optional[datetime.date] = Field(
        None,
        description= 'Earliest publish date filter, inclusive. Only use if explecitely specified',
    )

    latest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Latest publish date filter, exclusive. Only use if explicitly specified.",
    )
    min_length_sec: Optional[int] = Field(
        None,
        description="Minimum video length in seconds, inclusive. Only use if explicitly specified.",
    )
    max_length_sec: Optional[int] = Field(
        None,
        description="Maximum video length in seconds, exclusive. Only use if explicitly specified.",
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")
 

In [76]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a database query optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(TutorialSearch)
query_analyzer = prompt | structured_llm

In [77]:
query_analyzer.invoke({"question": "rag from scratch title"}).pretty_print()

content_search: rag from scratch
title_search: rag from scratch
min_view_count: 1000


In [78]:
query_analyzer.invoke(
    {"question": "videos on chat langchain published in 2023"}
).pretty_print()

content_search: chat langchain
title_search: 2023
earliest_publish_date: 2023-01-01
latest_publish_date: 2024-01-01


In [79]:
query_analyzer.invoke(
    {
        "question": "how to use multi-modal models in an agent, only videos under 5 minutes"
    }
).pretty_print()

content_search: multi-modal models agent
title_search: multi-modal models agent
max_length_sec: 300


https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/self_query/#constructing-from-scratch-with-lcel

In [ ]:
# pip install --upgrade --quiet  lark langchain-chroma

filter using the metadata of the document

In [80]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "science fiction",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())

In [95]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", 
        description="A 1-10 rating for the movie", 
        type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, 
    vectorstore, 
    document_content_description, 
    metadata_field_info, 
    verbose=True
)

In [93]:
# This example only specifies a filter
retriever.invoke("I want to watch a movie rated higher than 9.9")

[]

In [83]:
# This example specifies a query and a filter
retriever.invoke("Has Greta Gerwig directed any movies about women")

[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019})]

In [88]:
# This example specifies a composite filter
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

[Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979})]

In [89]:
# This example specifies a query and composite filter
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

[Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995})]